In [101]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

### im using dataset from this url 

#### https://www.kaggle.com/datasets/datasnaek/mbti-type

##### i hope you enjoy 

In [102]:
data = pd.read_csv('/kaggle/input/mbti-type/mbti_1.csv')

In [103]:
data

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...
...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,ENFP,'So...if this thread already exists someplace ...
8672,INTP,'So many questions when i do these things. I ...
8673,INFP,'I am very conflicted right now when it comes ...


In [104]:
def process_text (text):
    stop_words = stopwords.words('english')
    text = re.sub(r'\d+',' ',text)
    text = text.lower().strip()
    text = text.split()
    text = " ".join([word for word in text if word not in stop_words])
    return text

In [105]:
text = data['posts'].apply(process_text)

In [106]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(text)
text = tokenizer.texts_to_sequences(text)
max_length = np.max(list(map(lambda x : len(x),text)))

In [107]:
text = pad_sequences(text,maxlen=max_length,padding='post')

In [108]:
# types = sorted(list(data['type'].unique()))
# label = data['type'].apply(lambda x: types.index(x))

# 50% 

In [109]:
label = data['type'].apply(lambda x: 1 if x[0] == 'E' else 0)

In [111]:
text_train , text_test , label_train , label_test = train_test_split(text,label,train_size=0.7,random_state=13)

In [126]:
inputs = tf.keras.Input(shape=(max_length,))
embedding = tf.keras.layers.Embedding(
    input_dim=max_length
    ,output_dim=512
    ,input_length=10000
)(inputs)

gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU
    (
    units=256
    ,return_sequences=True
    )
)(embedding)

flatten = tf.keras.layers.Flatten()(gru)

outputs = tf.keras.layers.Dense(1,activation='sigmoid')(flatten)

model = tf.keras.Model(inputs,outputs)

model.compile(
    optimizer='adam'
    ,loss='binary_crossentropy'
    ,metrics='accuracy'
)

model.fit(
    text_train
    ,label_train
    ,validation_split=0.2
    ,batch_size=32
    ,epochs=20
    ,callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy'
        ,patience=3
        ,restore_best_weights=True
    )
)

Epoch 1/20
152/152 [==============================] - 40s 240ms/step - loss: 0.5577 - accuracy: 0.7624 - val_loss: 0.4382 - val_accuracy: 0.8239
Epoch 2/20
152/152 [==============================] - 31s 203ms/step - loss: 0.1346 - accuracy: 0.9564 - val_loss: 0.5201 - val_accuracy: 0.8066
Epoch 3/20
152/152 [==============================] - 28s 186ms/step - loss: 0.0142 - accuracy: 0.9984 - val_loss: 0.5711 - val_accuracy: 0.8123
Epoch 4/20
152/152 [==============================] - 26s 175ms/step - loss: 0.0035 - accuracy: 0.9996 - val_loss: 0.6572 - val_accuracy: 0.8173


In [127]:
model.evaluate(text_test,label_test)

82/82 [==============================] - 5s 58ms/step - loss: 0.4435 - accuracy: 0.8175


[0.44345322251319885, 0.8175182342529297]

In [128]:
true = np.array(label_test)
pred = np.squeeze(np.array(model.predict(text_test) >= 0.5 ,dtype=int))

82/82 [==============================] - 5s 55ms/step


In [133]:
print('REPORT!!!\n\n\n',classification_report(true,pred))

REPORT!!!


               precision    recall  f1-score   support

           0       0.83      0.96      0.89      1981
           1       0.75      0.35      0.48       622

    accuracy                           0.82      2603
   macro avg       0.79      0.66      0.68      2603
weighted avg       0.81      0.82      0.79      2603

